<a href="https://colab.research.google.com/github/1323216010/ColabNoteBook/blob/main/llm_classification_finetuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 登录kaggle并下载数据集

In [11]:
import kagglehub
kagglehub.login()
# {"username":"yan160100","key":"399e99783d0b56f99baf4f900899c9f3"}

llm_classification_finetuning_path = kagglehub.competition_download('llm-classification-finetuning')

print(llm_classification_finetuning_path)


/root/.cache/kagglehub/competitions/llm-classification-finetuning


# 定义函数

<a href="https://www.kaggle.com/code/carlacotas/llm-classification-finetuning?scriptVersionId=213890913" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [6]:
import re

import nltk
nltk.download('stopwords')
nltk.download('punkt_tab')
nltk.download('wordnet')
from nltk.corpus import stopwords

def which_winner(value):
    if value["winner_model_a"] == 1:
        #winner model a
        value["winner_model_b"] = 0
        value["winner_tie"] = 0
        return 0
    elif value["winner_model_b"] == 1:
        #winner model b
        return 1
    elif value["winner_tie"] == 1:
        #winner tie
        return 2
    return None

def preprocess_text(text):
    # convert text to lower case
    text = text.lower()
    # remove digits and special characters using regular expressions
    text = re.sub(r'\d+', '', text)
    text = re.sub(r'[^\w\s]', '', text)
    # tokenize the text
    text = nltk.word_tokenize(text)

    return text


def remove_stopwords(text):
    # remove stopwords
    stop_words = set(stopwords.words('english'))
    text_no_stopwords = [word for word in text if word not in stop_words]

    return text_no_stopwords


def lemmatization(text):
    lemmatizer = nltk.WordNetLemmatizer()
    lemmatizer_text = [lemmatizer.lemmatize(text) for text in text]

    return lemmatizer_text


def clean_text(text):
    # convert text to lower case, remove digits and special characters using regular expressions and remove stopwords combined together
    text = preprocess_text(text)
    filtered_text = remove_stopwords(text)
    lemmatizer_text = lemmatization(filtered_text)
    clean_text = ' '.join(lemmatizer_text)

    return clean_text

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


# 1.数据预处理

In [10]:
import numpy as np
import pandas as pd

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression



training = pd.read_csv(llm_classification_finetuning_path + '/train.csv')
test = pd.read_csv(llm_classification_finetuning_path + '/test.csv')

training["winner"] = training.apply(which_winner, axis=1)


training["prompt"] = training["prompt"].apply(clean_text)
training["response_a"] = training["response_a"].apply(clean_text)
training["response_b"] = training["response_b"].apply(clean_text)

test["prompt"] = test["prompt"].apply(clean_text)
test["response_a"] = test["response_a"].apply(clean_text)
test["response_b"] = test["response_b"].apply(clean_text)

print("Clean Cells with Text Data Complete")

Clean Cells with Text Data Complete


# 2.模型训练

In [8]:
all_training_text = pd.concat([training["prompt"], training["response_a"], training["response_b"]])
vectorizer = TfidfVectorizer(max_features=150)
vectorizer.fit(all_training_text)

temp_prompt = vectorizer.transform(training["prompt"])
temp_response_a = vectorizer.transform(training["response_a"])
temp_response_b = vectorizer.transform(training["response_b"])

train_y = training["winner"].values

# choosing "features"
train_X = np.concatenate((temp_prompt.toarray(), temp_response_a.toarray(), temp_response_b.toarray()), axis=1)

# Logistic Regression
model = LogisticRegression(max_iter=500, solver='saga')
model.fit(train_X, train_y)


Model winner prediction, probability [[0.17741276 0.25972753 0.5628597 ]
 [0.2186103  0.42894133 0.35244836]
 [0.34910118 0.35471831 0.29618051]]


# 3.输出

In [ ]:
all_training_text = pd.concat([training["prompt"], training["response_a"], training["response_b"]])
vectorizer = TfidfVectorizer(max_features=150)
vectorizer.fit(all_training_text)

temp_test_prompt = vectorizer.transform(test["prompt"])
temp_test_response_a = vectorizer.transform(test["response_a"])
temp_test_response_b = vectorizer.transform(test["response_b"])


#model prediction
test_X = np.concatenate((temp_test_prompt.toarray(), temp_test_response_a.toarray(), temp_test_response_b.toarray()), axis=1)
value_test_y_probabilities = model.predict_proba(test_X)
print('Model winner prediction, probability', value_test_y_probabilities) #winner model a | winner model b | winner tie


output = pd.DataFrame({'id': test.id,
                        'winner_model_a': value_test_y_probabilities[:, 0],
                        'winner_model_b': value_test_y_probabilities[:, 1],
                        'winner_tie': value_test_y_probabilities[:, 2]})
output.to_csv('submission.csv', index=False)